In [ ]:
import pretty_midi
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Dense, Input, Lambda, Concatenate, LSTM

from keras import backend as K

import tensorflow as tf
#import tensorflow_probability as tfp # for tf version 2.0.0, tfp version 0.8 is needed 
import numpy as np

import matplotlib.pyplot as plt
import csv
from sys import stdout
import random

# My code
from loading import *
from models import *

%matplotlib inline

In [2]:
print("TensorFlow version: {}".format(tf.__version__))
print("GPU is available: {}".format(tf.test.is_gpu_available()))

TensorFlow version: 2.0.0
GPU is available: False


In [3]:
file = 'maestro-v2.0.0/maestro-v2.0.0.csv'

In [4]:
# Call data class
data = DataObject(file, what_type = 'train', train_sec = 15, test_sec = 5, fs = 20)

# Create a batch class which we will iterate over
train_batch = Batch(data, 128, 4)

In [5]:
curr_batch = train_batch.data

In [6]:
model = simple_model(curr_batch, 15)

In [7]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer_context (InputLa (128, 59, 15, 88)         0         
_________________________________________________________________
Reshape_layer_1 (Lambda)     (7552, 15, 88)            0         
_________________________________________________________________
Encoder_lstm (LSTM)          (7552, 100)               75600     
_________________________________________________________________
Mean_representation_layer (L (128, 100)                0         
_________________________________________________________________
lambda_1 (Lambda)            (128, 150, 100)           0         
_________________________________________________________________
Decoder_lstm (LSTM)          [(128, 150, 88), (128, 88 66528     
Total params: 142,128
Trainable params: 142,128
Non-trainable params: 0
_____________________________________________________

In [ ]:
output = model.predict(curr_batch.context, steps=1)

In [ ]:
output.eval()

# Encoder

In [ ]:
lstm = LSTM(100)

In [ ]:
output = lstm(curr_batch.context)

In [ ]:
output.shape

In [ ]:
res_output = K.mean(tf.reshape(output, [128, 59, 100]), axis = -2)

In [ ]:
res_output.shape

In [ ]:
tile_output = tf.tile(tf.expand_dims(res_output, 1), [1,150,1])

In [ ]:
tile_output.shape

In [ ]:
K.mean(res_output, axis = -2).shape

In [ ]:
input_context_ = Input((None, 2), name="Input_layer_contxt_xy") # [num_pts, 2]
input_target_x = Input((None, 1), name="Input_layer_target_x")  # [num_pts, 1]

encoder = input_context_xy

In [ ]:
curr_batch.context.shape